<a href="https://colab.research.google.com/github/cbonnin88/Hospital_Admissions/blob/main/data_cleaning_and_EDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import polars as pl
import plotly.express as px
import gdown as gd

In [ ]:
# Loading raw data
url = 'https://drive.google.com/uc?id=1QNAup1SELVeOsXwOxlvpObIdw8n6vLNc'
gd.download(url,'raw_patients.csv',quiet=True)

'raw_patients.csv'

In [ ]:
df_patients = pl.read_csv('raw_patients.csv')

In [ ]:
df_patients.head()

patient_id,age,gender,dept_code,is_cmu
i64,i64,str,str,i64
1,29,"""M""","""Paris""",1
2,19,"""M""","""75""",0
3,94,"""M""","""Paris""",0
4,93,"""F""","""93""",0
5,20,"""F""","""75""",1


# **Cleaning Patient Data**

In [ ]:
# Deduplication Data
df_patients = df_patients.unique(subset=['patient_id'])

In [ ]:
# Standardize French Department Codes
dept_mapping = {
    'Paris': '75',
    '75001': '75',
    'Nanterre': '92'
}

In [ ]:
df_patients = df_patients.with_columns(
    pl.col('dept_code').replace(dept_mapping).alias('dept_code')
)

In [ ]:
# Handle Missing Gender
df_patients = df_patients.with_columns(
    pl.col('gender').fill_null('U')
)

In [ ]:
print(f'Cleaned Patients: {df_patients.shape}')

Cleaned Patients: (120000, 5)


# **Cleaning Hospital Stays**

In [ ]:
url='https://drive.google.com/uc?id=1dfwroRTisLNZvvrFJSPxVTOf_wYHTGTT'
gd.download(url,'raw_hospital_stay.csv',quiet=True)

df_stays = pl.read_csv('raw_hospital_stay.csv')

In [ ]:
df_stays.head()

stay_id,patient_id,hospital_id,admission_date,diagnosis_code,stay_cost,discharge_date
i64,i64,i64,str,str,f64,str
1,16840,13,"""2023-04-05T00:00:00.000000""","""E11.9""",7021.22,"""2023-04-15T00:00:00.000000"""
2,16291,33,"""2023-04-24T00:00:00.000000""","""J45.9""",7420.92,"""2023-05-04T00:00:00.000000"""
3,71675,48,"""2023-12-09T00:00:00.000000""","""J45.9""",13199.19,"""2023-12-19T00:00:00.000000"""
4,10568,6,"""2023-06-02T00:00:00.000000""","""J45.9""",6828.89,"""2023-06-12T00:00:00.000000"""
5,25866,14,"""2023-06-03T00:00:00.000000""","""E11.9""",3064.62,"""2023-06-13T00:00:00.000000"""


In [ ]:
# Converting Strings to Dates
df_stays = df_stays.with_columns([
    pl.col("admission_date").str.slice(0, 10).str.to_date("%Y-%m-%d"),
    pl.col("discharge_date").str.slice(0, 10).str.to_date("%Y-%m-%d")
])

In [ ]:
# Addings a Length of Stay column
df_stays = df_stays.with_columns(
    (pl.col('discharge_date') - pl.col('admission_date')).dt.total_days().alias('los_days')
)

In [ ]:
# Filter out any data errors (Discharge before Admission)
df_stays = df_stays.filter(pl.col("los_days") >= 0)

In [ ]:
df_stays.head()

stay_id,patient_id,hospital_id,admission_date,diagnosis_code,stay_cost,discharge_date,los_days
i64,i64,i64,date,str,f64,date,i64
1,16840,13,2023-04-05,"""E11.9""",7021.22,2023-04-15,10
2,16291,33,2023-04-24,"""J45.9""",7420.92,2023-05-04,10
3,71675,48,2023-12-09,"""J45.9""",13199.19,2023-12-19,10
4,10568,6,2023-06-02,"""J45.9""",6828.89,2023-06-12,10
5,25866,14,2023-06-03,"""E11.9""",3064.62,2023-06-13,10


# **Export for BigQuery**

In [ ]:
df_patients.write_csv('clean_patients.csv')
df_stays.write_csv('clean_hospital_stays.csv')
# Export the others as well
pl.read_csv('raw_hospitals.csv').write_csv('clean_hospitals.csv')
pl.read_csv('raw_pharmacy_claims.csv').write_csv('clean_pharmacy_claims.csv')

# **Readmission Frequency Analysis**

In [ ]:
readmission_freq = (
    df_stays
    .group_by('patient_id')
    .agg(pl.count('stay_id').alias('stay_count'))
    .group_by('stay_count')
    .agg(pl.count('patient_id').alias('number_of_patients'))
    .sort('stay_count')
)

display(readmission_freq)

stay_count,number_of_patients
u32,u32
1,44084
2,21896
3,7476
4,1863
5,353
6,67
7,11


In [ ]:
fig_readmission = px.bar(
    readmission_freq.to_pandas(), # Plotly works better with pandas
    x='stay_count',
    y='number_of_patients',
    title='Frequency of Patient Hospitalizations',
    labels={'stay_count':'Number of Stays','number_of_patients':'Number of Patients'},
    text_auto=True,
    color_discrete_sequence=['#3498db']
)

fig_readmission.show()

# **Average Cost by Age Group **

In [ ]:
# Aggregate average Cost
age_analysis = (
    df_stays.join(df_patients, on='patient_id')
    .with_columns(
        pl.when(pl.col('age') < 30).then(pl.lit('18-29'))
        .when(pl.col('age') < 45).then(pl.lit('30-44'))
        .when(pl.col('age') < 65).then(pl.lit('45-64'))
        .otherwise(pl.lit('65+'))
        .alias('age_group')
    )
    .group_by('age_group')
    .agg(pl.col('stay_cost').mean().round(2).alias('avg_stay_cost'))
    .sort('age_group')
)

display(age_analysis)

age_group,avg_stay_cost
str,f64
"""18-29""",7770.08
"""30-44""",7732.25
"""45-64""",7755.64
"""65+""",7758.43


In [ ]:
fig_age_cost = px.pie(
    age_analysis.to_pandas(),
    values='avg_stay_cost',
    names='age_group',
    title='Proportion of Average Stay Cost by Age Group',
    hole=0.4,
    color_discrete_sequence=px.colors.sequential.RdBu
)

fig_age_cost.show()

# **Total Healthcare Expenditure by Department**

In [ ]:
# Aggregation: Join and sum costs by dept_code
dept_analysis = (
    df_stays.join(df_patients, on='patient_id')
    .group_by('dept_code')
    .agg(pl.col('stay_cost').sum().alias('total_spend_euro'))
    .sort('total_spend_euro',descending=True)
)

display(dept_analysis)

dept_code,total_spend_euro
str,f64
"""75""",3.9806e8
"""92""",2.6664e8
"""94""",1.3311e8
"""93""",1.3273e8


In [ ]:
fig_dept = px.bar(
    dept_analysis.to_pandas(),
    x='dept_code',
    y='total_spend_euro',
    title='Total Healthcare Spend by French Department Code',
    labels={'dept_code':'Department','total_spend_euro':'Total Spend (€)'},
    color='total_spend_euro',
    color_continuous_scale='Viridis'
)

fig_dept.show()

# **Patient Age Distribution**

In [ ]:
fig_age = px.histogram(
    df_patients.to_pandas(),
    x='age',
    nbins=20,
    title='Age Distribution of Patients (France ARS)',
    color_discrete_sequence=['#2ecc71'],
    labels={'count':'Number of Patients','age':'Age'}
)

fig_age.show()

# **Cost by Diagnosis (CIM-10)**

In [ ]:
cost_summary = df_stays.group_by('diagnosis_code').agg(pl.col('stay_cost').mean().round(2).alias('avg_cost'))

display(cost_summary)

diagnosis_code,avg_cost
str,f64
"""J44.0""",7730.85
"""E11.9""",7753.66
"""J45.9""",7780.24
"""I10""",7752.85


In [ ]:
fig_cost = px.bar(
    cost_summary.to_pandas(),
    x='diagnosis_code',
    y='avg_cost',
    title='Average Stay Cost by CIM-10 Diagnosis Code',
    labels={'diagnosis_code':'Diagnosis','avg_cost':'Avg Cost (€)'},
    color='avg_cost',
    color_continuous_scale='Viridis'
)

fig_cost.update_layout(coloraxis_showscale=False)
fig_cost.show()